In [1]:
import keras
import numpy as np
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GRU,Input
from keras.layers.embeddings import Embedding
import matplotlib as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.cross_validation import train_test_split
from tensorflow.contrib import learn
import os

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s ", string)
    string = re.sub(r"\'ve", " have ", string)
    string = re.sub(r"n\'t", " not ", string)
    string = re.sub(r"\'re", " are ", string)
    string = re.sub(r"\'d" , " would ", string)
    string = re.sub(r"\'ll", " will ", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " \? ", string)
#   string = re.sub(r"[^\d\.]","",string)
#   string = re.sub(r"[a-zA-Z]{4,}", "", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip() if TREC else string.strip().lower()

In [3]:
df=pd.read_csv('essays.csv',encoding='latin-1')
df.head()

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",n,y,y,n,y
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",n,n,y,n,n
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,n,y,n,y,y
3,1997_568848.txt,I can't believe it! It's really happening! M...,y,n,y,y,n
4,1997_688160.txt,"Well, here I go with the good old stream of co...",y,n,y,n,y


In [4]:
len(df)

2467

In [5]:
raw=df['TEXT']
#raw[0]

In [6]:
emotionfile=pd.read_csv('Emotion_Lexicon.csv')
emotionfile.head()

,Words,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Charged
0,march,0,0,0,0,0,0,1,0,0,0,1
1,august,0,0,0,0,0,0,1,0,0,0,1
2,ago,0,0,0,0,0,0,0,0,0,0,0
3,mar,0,0,0,0,0,1,0,0,0,0,1
4,vie,0,0,0,0,0,0,0,0,0,0,0


In [7]:
emontial=emotionfile[emotionfile['Charged']!=0]['Words'].tolist()
#emontial

#emotionfile['sum']=emotionfile.sum(axis=1)
#emontial=emotionfile[emotionfile['sum']!=0]['Words'].tolist()
#emontial

In [19]:
doc=[]
sentence_length=[]
unemontial=[]

In [20]:
len(emontial)

6468

In [22]:
for i in range(len(raw)):
    essay=re.split(r'[.?]',raw[i])
    sentences=[]
    for sent in essay:
        sent=clean_str(sent)
        sent=sent.split(' ')
        if(len(sent)>150):
            newsent=[]
            splits=int(np.floor(len(sent)/20))
            for index in range(splits):
                            newsent.append(' '.join(sent[index*20:(index+1)*20]))
            if len(sent)>splits*20:
                            newsent.append(' '.join(sent[splits*20:]))
            for s in newsent:
                n=0
                s=s.split(' ')
                for word in s:
                    if word in emontial:
                        n=n+1
                if n!=0:
                    sentence_length.append(len(s))
                    s=' '.join(s)
                    sentences.append(s)
                    
        else:
            n=0
            for word in sent:
                if word in emontial:
                    n=n+1
            if n!=0:
                sentence_length.append(len(sent))
                sent=' '.join(sent)
                sentences.append(sent)
    if sentences:
        doc.append(sentences)
    else:
        unemontial.append(sentences)

In [26]:
doc[-1]

["he wants to go to a hotel , but i know i have to babysit and i'm not sure exactly how i will tell my dad to pick us up from here",
 'my birthday is this weekend , and it does not really even feel like it',
 'i do not really mind , but i have a feeling brady probably wo not do much for it',
 'he says he will pay for the hotel , but i do not really want that for my birthday',
 "i'ts just not htat important to me",
 'i feel guilty today for lying to my dad about gettin gout of the tip program',
 'i know that sometimes i have to lie about things like that in order for them to understand',
 "i have not heard from my mom so i'm not sure if she 's mad at me or not",
 'i wish brady would pay more attention',
 'he tries so hard , but it does not seem like he has the chance to do so',
 "i'm feeling that although i do not weigh that much my weight is getting out of control",
 'kristen and i went shopping yesterday',
 'i really do not plan on shopping anymore',
 "i'm surprised i did not get one"

In [30]:
raw[2466]

"I have just gotten off the phone with brady. I'm trying to decide what exacly we will do  this weekend. he wants to go to a hotel, but I know I have to babysit and I'm not sure exactly  how I will tell my dad to pick us up from here. My birthday is this weekend, and it doesn't really  even feel like it. I don't really mind, but I have a feeling Brady probably won't do much for it. I  almost feel like I'm always doing something for him. I should, I really care about him. He says  he will pay for the hotel, but I don't really want that for my birthday. I'ts just not htat important  to me. I feel guilty today for lying to my dad about gettin gout of the TIP program. I know that  sometimes I have to lie about things like that in order for them to understand. I haven't heard  from my mom so I'm not sure if she's mad at me or not. She probably doesn't care, but when  she does find out she will freak. I wish Brady would pay more attention. He tries so hard, but it  doesn't seem like he has t

In [41]:
max_sentencelength=max(sentence_length)
max_sentencelength

149

In [45]:
b=[]
for essay in doc:
    for sentence in essay:
        b.append(sentence)

In [46]:
len(b)

73334

In [49]:
b[0]

'well , right now i just woke up from a mid day nap'

In [50]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_sentencelength)
vocab_processor.fit_transform(b)
np.array(list(vocab_processor.transform(doc[0])))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


array([[  1,   2,   3, ...,   0,   0,   0],
       [ 13,  14,  15, ...,   0,   0,   0],
       [  4,  30,  31, ...,   0,   0,   0],
       ...,
       [ 32, 127, 123, ...,   0,   0,   0],
       [  1,  13,  14, ...,   0,   0,   0],
       [ 59,   4, 251, ...,   0,   0,   0]])

In [65]:
data=doc.copy()
data[0][0]

'well , right now i just woke up from a mid day nap'

In [66]:
for i in range(len(data)):
    data[i]=np.array(list(vocab_processor.transform(data[i])))

In [67]:
data=np.array(data)
data

array([array([[  1,   2,   3, ...,   0,   0,   0],
       [ 13,  14,  15, ...,   0,   0,   0],
       [  4,  30,  31, ...,   0,   0,   0],
       ...,
       [ 32, 127, 123, ...,   0,   0,   0],
       [  1,  13,  14, ...,   0,   0,   0],
       [ 59,   4, 251, ...,   0,   0,   0]]),
       array([[  1,  50, 255, ...,   0,   0,   0],
       [  4, 184,  22, ...,   0,   0,   0],
       [134, 261, 262, ...,   0,   0,   0],
       ...,
       [367,   1,  95, ...,   0,   0,   0],
       [186, 106, 107, ...,   0,   0,   0],
       [ 46, 166, 186, ...,   0,   0,   0]]),
       array([[ 39,  66, 370, ...,   0,   0,   0],
       [  4, 236, 376, ...,   0,   0,   0],
       [383, 383, 383, ...,   0,   0,   0],
       ...,
       [496, 281, 538, ...,   0,   0,   0],
       [140,   9, 397, ...,   0,   0,   0],
       [186,  22, 125, ...,   0,   0,   0]]),
       ...,
       array([[   4,  193,  663, ...,    0,    0,    0],
       [1211,   39, 9412, ...,    0,    0,    0],
       [  39,  147,   71, 

In [68]:
numsentences=[]
for i in range(len(data)):
    numsentences.append(len(data[i]))
longistsent=max(numsentences)
print(longistsent)
numsentences[:10]

129


[27, 51, 36, 23, 36, 54, 41, 15, 1, 25]

In [69]:
for i in range(len(data)):
    ess=data[i]
    if len(ess)<129:
        zeros=np.zeros([129-len(ess),149])
        data[i]=np.concatenate((ess,zeros),axis=0)

In [70]:
data[0].shape

(129, 149)

In [71]:
vocab_dict = vocab_processor.vocabulary_._mapping

In [72]:
len(vocab_dict)

24781

In [56]:
embeddings_index = {}
f = open(os.path.join('/Users/ZP/Downloads/CS242/final project/glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [57]:
embedding_matrix = np.zeros((len(vocab_dict), 300))
for word, i in vocab_dict.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [62]:
embedding_matrix.shape
embedding_matrix[1]

array([-1.35089993e-01,  3.59070003e-01,  1.45300001e-01, -1.28289998e-01,
       -5.57250008e-02,  4.01080012e-01, -9.40980017e-02,  2.30649993e-01,
        6.72950000e-02, -1.94309998e+00,  2.86509991e-01, -3.13659990e-03,
       -4.01000008e-02,  2.88390011e-01,  3.99289988e-02, -8.70169979e-03,
       -5.51980019e-01,  1.09080002e-01,  1.52199998e-01,  1.69179998e-02,
       -3.23860012e-02,  2.14800000e-01,  2.85769999e-01,  1.69420004e-01,
       -3.66019994e-01,  5.65330014e-02, -3.63910012e-02,  5.48620000e-02,
       -1.85980007e-01,  2.34190002e-01,  1.85440004e-01,  3.51570010e-01,
       -2.84969985e-01,  1.27250001e-01, -7.76780009e-01,  4.64040004e-02,
        2.24399999e-01,  2.63660014e-01,  6.99499995e-03, -1.83129996e-01,
       -1.81970000e-01, -3.68809998e-01, -4.36020009e-02, -1.31569996e-01,
        1.39960006e-01,  3.40290010e-01,  3.33579987e-01,  4.57659990e-01,
       -2.21759994e-02,  4.24529999e-01, -5.87569997e-02, -1.77359998e-01,
        1.80380002e-01, -

In [ ]:
def make_essay(sentences,essay_length)

In [418]:
def make_sentence(sentence,sentence_length):
    n=0
    sentence.split('')
    for word in sentence:
        if word in emontial:
            n=n+1
    if n!=0:
        #print(len(sentence))
        while(len(sentence)<sentence_length):
            index=sentence_length*np.random.rand(1)[0]
            #print('raw index',index)
            index=int(np.floor(index))-1
            #print('index',index)
            #print('length',len(sentence))
            sentence.append(sentence[index])
            #print('length after add',len(sentence))
        s=' '.join(sentence)
        return s
    else:
        return ''
        
        
    
            

In [415]:
int(np.floor(6*np.random.rand(1)[0]))

4

In [420]:
raw.head()

0    Well, right now I just woke up from a mid-day ...
1    Well, here we go with the stream of consciousn...
2    An open keyboard and buttons to push. The thin...
3    I can't believe it!  It's really happening!  M...
4    Well, here I go with the good old stream of co...
Name: TEXT, dtype: object

In [422]:
len(doc)

2435

In [424]:
doc[0][0]

'well , right now i just woke up from a mid day nap'

In [425]:
len(raw)

2467

In [426]:
type(raw)

pandas.core.series.Series

In [427]:
doc[0]

['well , right now i just woke up from a mid day nap',
 "it 's sort of weird , but ever since i moved to texas , i have had problems concentrating on things",
 'i remember starting my homework in 10th grade as soon as the clock struck 4 and not stopping until it was done',
 'but when i moved here , the homework got a little more challenging and there was a lot more busy work , and so i decided not to spend hours doing it , and just getting by',
 "but the thing was that i always paid attention in class and just plain out knew the stuff , and now that i look back , if i had really worked hard and stayed on track the last two years without getting lazy , i would have been a genius , but hey , that 's all good",
 "it 's too late to correct the past , but i do not really know how to stay focused n the future",
 "for me it would be easier there , but alas , i'm living at home under the watchful eye of my parents and a little nagging sister that just nags and nags and nags",
 "another thing i